In [1]:
!pip install praw
!pip install gTTS
!pip install pydub
!pip install pandas

In [39]:
!pip install p


^C


In [3]:
import praw
import json
# from prawcore.exceptions import ResponseException
from pydub import AudioSegment
from pydub.playback import play
from gtts import gTTS
import numpy as np
import pandas as pd

# reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
reddit_object = praw.Reddit(
    client_id="HDdI1LLvE7uDANi3oQ2teQ",
    client_secret="oDoTRKQhuCRVHmqgb5RyjyFsyQAYqw",
    user_agent="testing by u'/JuggernautBorn5378 "
    # username="JuggernautBorn5378",
    # password="kanakshat"
)


In [7]:
search = str(input("What you want to search...."))
# typ = input("Enter what type of content you want to extract hot, top, new, controversial, giled....")
number_of_post = int(input("How many post you want...."))
search_hot = reddit_object.subreddit('all').search(search, sort='relevance', time_filter='all', limit=10000)
# search = reddit_object.subreddit(f'{search}')
# search_hot = search.top(limit = 100)
result_title_posts = []
result_url_posts = []
result_title = []
result_url = []
video_url = []
audio_url = []
posts = []
for i in search_hot:
    temp = dict()
    # if len(set(result_title)) == number_of_post:
    #     break
    temp['is_video'] = i.is_video
    temp['i.media'] = True if i.media and 'reddit_video' in i.media else False
    temp['title'] = i.title
    temp['reddit_post_url'] = i.url
    temp['Number_of_likes'] = i.score
    temp['Number_of_comments'] = i.num_comments
    temp['video_url'] = i.media['reddit_video']['fallback_url'] if i.media and 'reddit_video' in i.media else "None"
    temp['audio_url'] = i.media['reddit_video']['fallback_url'].split("DASH_")[0] + "DASH_AUDIO_128.mp4" if i.media and 'reddit_video' in i.media else "None"
    posts.append(temp)
data = pd.DataFrame(posts)
data = data[data['is_video']==True]
data['Number_of_likes'] = data['Number_of_likes'].astype(int)
data = data.sort_values(by = ['Number_of_likes'], ascending=False)
    # if i.is_video ==True:
    #     if i.media and 'reddit_video' in i.media:
    #         # print(dir(i))

    #         video = i.media['reddit_video']['fallback_url']
    #         video_url.append(video)
    #         audio_url.append(video.split("DASH_")[0] + "DASH_AUDIO_128.mp4")
    #     result_title.append(i.title)
    #     result_url.append(i.url)
    # if i.is_video ==False:
    #     result_title_posts.append(i.title)
    #     result_url_posts.append(i.url)

print(data)
data = data[(data['video_url']!= 'None') & (data['title'].str.contains(f'{search}', case=False))]


data = data.drop_duplicates(subset = 'title').reset_index(drop=True)
data = data[:10]
print(data)

if data.shape[0]>0:
    text = f"Hello Guys,here are the top {search} Videos for you "

    tts = gTTS(text)
    tts.save(f"output.mp3")
    audio = AudioSegment.from_mp3("output.mp3")
    # play(audio)
    title = data['title'].unique().tolist()
    result_url = data['reddit_post_url'].unique().tolist()
    video_url = data['video_url'].unique().tolist()
    audio_url = data['audio_url'].unique().tolist()
    print("-"*10, "Reddit Posts", result_url)
    print(title)
    print(video_url)
    print(audio_url)

    import os
    import requests
    from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips

    def download_and_merge_videos(video_urls, audio_urls, output_folder):
        # Create the output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Loop through the provided video and audio URLs
        for i, (video_url, audio_url) in enumerate(zip(video_urls, audio_urls), start=1):
            # Download video content
            video_response = requests.get(video_url)
            video_filename = os.path.join(output_folder, f"video_{i}.mp4")

            if video_response.status_code == 200:
                with open(video_filename, 'wb') as video_file:
                    video_file.write(video_response.content)
                print(f"Video downloaded and saved as {video_filename}")
            else:
                print(f"Failed to download video from {video_url}. Status code: {video_response.status_code}")
                continue

            # Download audio content
            audio_response = requests.get(audio_url)
            audio_filename = os.path.join(output_folder, f"audio_{i}.mp3")

            if audio_response.status_code == 200:
                with open(audio_filename, 'wb') as audio_file:
                    audio_file.write(audio_response.content)
                print(f"Audio downloaded and saved as {audio_filename}")
            else:
                print(f"Failed to download audio from {audio_url}. Status code: {audio_response.status_code}")
                continue

            # Merge video and audio
            video_clip = VideoFileClip(video_filename)
            audio_clip = AudioFileClip(audio_filename)
            video_clip = video_clip.set_audio(audio_clip)

            # Write the final video file
            final_output_filename = os.path.join(output_folder, f"final_video_{i}.mp4")
            try:
                video_clip.write_videofile(final_output_filename, codec='libx264', audio_codec='aac')
                print(f"Final video saved as {final_output_filename}")
            except Exception as e:
                print(f"Error writing final video file: {e}")

            # Optionally, you can delete the individual video and audio files if needed
            # os.remove(video_filename)
            # os.remove(audio_filename)

    # Example usage
    video_urls = video_url
    audio_urls = audio_url
    output_folder = "final_videos"
    download_and_merge_videos(video_urls, audio_urls, output_folder)



    def merge_audio_and_video(audio_path, video_path, output_path):
        # Load the audio and video clips
        audio_clip = AudioFileClip(audio_path)
        video_clip = VideoFileClip(video_path)

        # Set the audio of the video to the loaded audio clip
        video_clip = video_clip.subclip(0, audio_clip.duration)
        video_clip = video_clip.set_audio(audio_clip)
        # Write the merged video to the output file
        video_clip.write_videofile(output_path, codec='libx264', audio_codec='aac', fps=24)


    audio_file_path = 'output_audio.mp3'  # Replace with the path to your audio file
    video_file_path = 'final_videos/video_1.mp4'  # Replace with the path to your video file
    output_file_path = 'final_videos/final_output_video.mp4'  # Replace with the desired output path

    merge_audio_and_video(audio_file_path, video_file_path, output_file_path)
else:
    print("No Data Found")

print("---------------------------done-----------------------------------------------------")

    is_video  i.media                                              title  \
0       True     True                Badal barsa bijuli sawan ko paani 💦   
38      True     True          Ham ladke hai hamare sath esa hi hota hai   
18      True     True        Vnit nagpur mai baarish se hostel doob gaya   
20      True     True                     Papa player love from Nepal 🇳🇵   
19      True     True                                       Naya cancer🤡   
11      True     True                              Badal barsa bicholi!!   
26      True     True                                        Band Karo 🙏   
36      True     True                  Sunai dera mujhe to dera hai🕺🕺🕺🕺🕺   
40      True     True                                              Badal   
39      True     True                                              Riyal   
10      True     True                                Badal Barse Bijuli    
13      True     True                                 Badal Barsa bijlui   
2       True

t:  42%|████▏     | 180/429 [07:58<00:33,  7.51it/s, now=None]

Moviepy - Building video final_videos\final_video_1.mp4.
MoviePy - Writing audio in final_video_1TEMP_MPY_wvf_snd.mp4


t:  42%|████▏     | 180/429 [07:58<00:33,  7.51it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_videos\final_video_1.mp4



...

Moviepy - Building video final_videos/final_output_video.mp4.
MoviePy - Writing audio in final_output_videoTEMP_MPY_wvf_snd.mp4


t:  42%|████▏     | 180/429 [10:18<00:33,  7.51it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_videos/final_output_video.mp4



t:  42%|████▏     | 180/429 [10:21<00:33,  7.51it/s, now=None]

Moviepy - Done !
Moviepy - video ready final_videos/final_output_video.mp4
---------------------------done-----------------------------------------------------


In [ ]:
video_url = 'https://v.redd.it/qxmuk5rp6t2c1/DASH_720.mp4?source=fallback'
audio_url = video_url.split("DASH_")[0] + "audio"
audio_url_alternatvie = video_url.split("DASH_")[0] + "DASH_AUDIO_128.mp4"
print(audio_url, audio_url_alternatvie)

In [ ]:
!pip install --upgrade --force-reinstall pandas

In [ ]:
from moviepy.editor import TextClip, CompositeVideoClip

# Create a text clip
text_clip = TextClip("", fontsize=70, color='white', bg_color='black')

# Set the duration of the text clip (in seconds)
text_clip = text_clip.set_duration(5)

# Create a composite video clip
video_clip = CompositeVideoClip([text_clip])

# Write the video file
video_clip.write_videofile('output_video.mp4', codec='libx264', audio_codec='aac', fps=24)

In [4]:
import numpy as np
import cv2
from moviepy.editor import VideoFileClip, CompositeVideoClip

# Create a blank video canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Load the AI-powered man image
ai_man_image = cv2.imread('ai_man.jpeg')

# Resize the AI-powered man image to fit the canvas
resized_ai_man_image = cv2.resize(ai_man_image, (200, 200))

# Add the AI-powered man image to the canvas
canvas[100:300, 220:420] = resized_ai_man_image

# Create a text overlay
text = "This is some text."
font = cv2.FONT_HERSHEY_SIMPLEX
text_size = cv2.getTextSize(text, font, 1, 2)[0]
text_x = (canvas.shape[1] - text_size[0]) // 2
text_y = canvas.shape[0] - 50

cv2.putText(canvas, text, (text_x, text_y), font, 1, (255, 255, 255), 2)

# Convert the canvas to a VideoFileClip object
cv2.imwrite('canvas.png', canvas)
canvas_video = VideoFileClip('canvas.png')

# canvas_video = VideoFileClip(canvas)

# Create a composite video clip with the canvas video and a background video
background_video = VideoFileClip('background.mp4')
composite_video = CompositeVideoClip([canvas_video, background_video.set_opacity(0.5)])

# Export the composite video
composite_video.write_videofile('output.mp4')


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


                                                                   

                                                  
chunk:   1%|          | 1/116 [03:33<6:48:25, 213.09s/it, now=None]
                                                                   

                                                            
chunk:   1%|          | 1/116 [03:33<6:48:26, 213.10s/it, now=None]


Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp4


Error writing video file: index -100001 is out of bounds for axis 0 with size 0


Video downloaded and saved as merged_videos\video_1.mp4
Audio downloaded and saved as merged_videos\audio_1.mp3


                                                                   

chunk:   1%|          | 1/116 [1:22:27<158:02:56, 4947.63s/it, now=None]
                                                                        

chunk:   1%|          | 1/116 [1:22:27<158:02:58, 4947.64s/it, now=None]


Moviepy - Building video merged_videos\final_video_1.mp4.
MoviePy - Writing audio in final_video_1TEMP_MPY_wvf_snd.mp4


                                                                        

chunk:   1%|          | 1/116 [1:22:28<158:05:01, 4948.71s/it, now=None]
                                                                        

chunk:   1%|          | 1/116 [1:22:28<158:05:03, 4948.72s/it, now=None]


MoviePy - Done.
Moviepy - Writing video merged_videos\final_video_1.mp4



...

Moviepy - Building video merged_videos\final_video_4.mp4.
MoviePy - Writing audio in final_video_4TEMP_MPY_wvf_snd.mp4


                                                                        

chunk:   1%|          | 1/116 [1:25:34<164:00:25, 5134.14s/it, now=None]
                                                                        

chunk:   1%|          | 1/116 [1:25:34<164:00:27, 5134.15s/it, now=None]


MoviePy - Done.
Moviepy - Writing video merged_videos\final_video_4.mp4



                                                                        

chunk:   1%|          | 1/116 [1:25:35<164:03:43, 5135.85s/it, now=None]
                                                                        

chunk:   1%|          | 1/116 [1:25:35<164:03:44, 5135.87s/it, now=None]


Moviepy - Done !
Moviepy - video ready merged_videos\final_video_4.mp4
Final video saved as merged_videos\final_video_4.mp4
